# PyTorch in 1 Hour – Hands‑on Tutorial
*Generated on 2025-07-06*

This notebook is a distilled, hands‑on companion to **Sebastian Raschka’s** article *PyTorch in One Hour: From Tensors to Multi‑GPU Training*. Follow the numbered sections to explore core PyTorch concepts and run bite‑sized code examples yourself.

📚 **Reference**: https://sebastianraschka.com/teaching/pytorch-1h/

In [ ]:
import torch
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Compute device:', device)

## 1. What is PyTorch?
At its heart, **PyTorch** provides:
1. **Tensors** – N‑dimensional arrays with GPU acceleration.
2. **Autograd** – automatic differentiation for building and training neural networks.
3. **`torch.nn` high‑level API** – a library of pre‑built layers, loss functions, and utilities.

The dynamic computation graph makes debugging intuitive because operations run **eagerly** like regular Python code.

In [ ]:
# Toy example: a+b with autograd
a = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)
c = a * b + b
c.backward()
print('dc/da =', a.grad)
print('dc/db =', b.grad)

## 2. Working with Tensors
### 2.1 Creating tensors

In [ ]:
x = torch.zeros((2, 3))
y = torch.ones((2, 3))
z = torch.rand((2, 3))
print(x)
print(y)
print(z)

### 2.2 Tensor attributes & basic ops

In [ ]:
t = torch.arange(12).reshape(3, 4)
print('Shape:', t.shape)
print('Datatype:', t.dtype)
print('Device:', t.device)

# Broadcasting addition
print(t + 10)

## 3. Automatic Differentiation (Autograd)
`requires_grad` tells PyTorch to track operations on a tensor. After the forward pass, call `.backward()` to populate `.grad`.

In [ ]:
w = torch.tensor([2.0, 3.0], requires_grad=True)
output = (w ** 2).sum()
output.backward()
print('Gradient:', w.grad)

## 4. Building Neural Networks with `nn.Module`
A minimal two‑layer MLP example:

In [ ]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

model = Net().to(device)
print(model)

## 5. Training Loop Essentials
Pseudo‑training loop skeleton:

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# dummy one‑batch train step
images = torch.rand((32, 784), device=device)
labels = torch.randint(0, 10, (32,), device=device)

logits = model(images)
loss = loss_fn(logits, labels)
loss.backward()
optimizer.step()
optimizer.zero_grad()
print('Loss:', loss.item())

## 6. Device Management & Mixed Precision

In [ ]:
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()
with autocast():
    logits = model(images)
    loss = loss_fn(logits, labels)
scaler.scale(loss).backward()
scaler.step(optimizer)
scaler.update()

## 7. Multi‑GPU Training Basics
Use `torch.nn.DataParallel` for quick mirroring on multiple GPUs or `torch.nn.parallel.DistributedDataParallel` for large‑scale jobs.

In [ ]:
# Quick DataParallel wrapper (works if >1 GPUs detected)
if torch.cuda.device_count() > 1:
    print('GPUs:', torch.cuda.device_count())
    model = nn.DataParallel(model)


## 8. Saving & Loading Models

In [ ]:
torch.save(model.state_dict(), 'model.pt')
# Later:
loaded_model = Net().to(device)
loaded_model.load_state_dict(torch.load('model.pt'))

## 9. End‑to‑End Mini‑Project
Try training the above MLP on **Fashion‑MNIST** or **MNIST**. Use torchvision datasets & transforms. Code omitted for brevity.

## 10. Next Steps & Resources
- Official PyTorch docs <https://pytorch.org/docs>
- PyTorch Forums <https://discuss.pytorch.org>
- Sebastian Raschka’s book *Machine Learning with PyTorch and Scikit‑Learn*